In [14]:
%reset -f

In [1]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from datetime import datetime

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

# IMPORT BQ NEW WAY

In [4]:
import MyFunc
# from MyFunc import cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *
# cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
def split_indo_foreign_text(input_text):
    input_text=str(input_text)
    if len(input_text.split('(Asli)'))>1 :
        result=input_text.replace('(Diterjemahkan oleh Google)','').split('(Asli)')
        indo_text=result[0].strip()
        foreign_text=result[1].strip()
    elif len(input_text.split('(Diterjemahkan oleh Google)'))>1 :
        result=input_text.replace('(Asli) ','').split('(Diterjemahkan oleh Google)')
        indo_text=result[1].strip()
        foreign_text=result[0].strip()
    else:
        indo_text=input_text
        foreign_text=None
        
    return pd.Series([indo_text,foreign_text])

def cleaning_reviews(data):
    new_columns=[]
    for column in data.columns :
        column=column.lower()
        new_columns.append(column.replace(' ','_'))
        try:
            data[column]=data[column].str.replace(';','').str.replace('"','').str.replace('\n',' ').str.replace('\r','').str.replace('\t','')
        except:
            pass

    data.columns=new_columns
    data[['indo_text','foreign_text']]=data.review_text.parallel_apply(split_indo_foreign_text)
    new_columns=list(data.columns[:15])+['indo_text','foreign_text']+list(data.columns[15:-2])
    data=data[new_columns]
    return data

In [7]:
%%time

query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_reviews_all` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1
CPU times: user 61.1 ms, sys: 24.3 ms, total: 85.4 ms
Wall time: 1.31 s


In [8]:
%%time
import os
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

scrappe_date='20231107'

path = 'outcrapper_raw/reviews/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

# df_temp=pd.read_csv('results/clear_reviews_table.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     my_schema.append(bigquery.SchemaField(column, "STRING"))

for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.review_id!='__NO_REVIEWS_FOUND__']
        
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_reviews(data)

        # df_temp=pd.read_csv('results/clear_reviews_table.csv',sep=';', nrows=1)
        my_schema=[]
        save_columns=[]
        for column in df_temp.columns :
            if column in data.columns :
                save_columns.append(column)
                my_schema.append(bigquery.SchemaField(column, "STRING"))
            
        
        data[save_columns].to_csv('results/temp_upload_review.csv',sep=';',index=False)

        #############################################################################################
        #BQ DATALAKE HOREGO
        src_path_file='results/temp_upload_review.csv'
        target_path_file=f"scrapping_results/reviews/{file.replace('.json','.csv')}"
        bucket_name='horego-bq'
        upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        table_schema = "horego_datalake_dev"
        table_name = "outcrapper_google_reviews_all"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/reviews/{file.replace('.json','.csv')}"
        create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        
        #############################################################################################
        #############################################################################################
        #BQ DATALAKE CASHBAC
        bucket_name='cashbac_datalake'
        src_path_file='results/temp_upload_review.csv'
        target_path_file=f"scrapping_results/reviews/{file.replace('.json','.csv')}"
        upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

        table_schema = "cashbac_datalake_prod"
        table_name = "outcrapper_google_reviews_all"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/reviews/{file.replace('.json','.csv')}"
        create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
    
        #############################################################################################
 
del data
print('DONE')

20231107033642cd2f_reviews_horego_placelist_reupload.json
Uploading results/temp_upload_review.csv
Path in GCS: horego-bq/scrapping_results/reviews/20231107033642cd2f_reviews_horego_placelist_reupload.csv
table on : horego_datalake_dev.outcrapper_google_reviews_all
Uploading results/temp_upload_review.csv
Path in GCS: cashbac_datalake/scrapping_results/reviews/20231107033642cd2f_reviews_horego_placelist_reupload.csv
table on : cashbac_datalake_prod.outcrapper_google_reviews_all
20231107063826fd41_reviews_horego_placelist_bandung.json
Uploading results/temp_upload_review.csv
Path in GCS: horego-bq/scrapping_results/reviews/20231107063826fd41_reviews_horego_placelist_bandung.csv
table on : horego_datalake_dev.outcrapper_google_reviews_all
Uploading results/temp_upload_review.csv
Path in GCS: cashbac_datalake/scrapping_results/reviews/20231107063826fd41_reviews_horego_placelist_bandung.csv
table on : cashbac_datalake_prod.outcrapper_google_reviews_all
DONE
CPU times: user 2.04 s, sys: 392